### Setup and Dependencies Installation

In [1]:
!pip install pdfminer.six langchain sentence-transformers transformers accelerate peft datasets torch bitsandbytes


  Using cached pdfminer_six-20250506-py3-none-any.whl (5.6 MB)
  Using cached langchain-0.3.27-py3-none-any.whl (1.0 MB)
  Using cached sentence_transformers-5.0.0-py3-none-any.whl (470 kB)
  Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
  Using cached cryptography-45.0.5-cp311-abi3-win_amd64.whl (3.4 MB)
  Using cached langchain_core-0.3.72-py3-none-any.whl (442 kB)
  Using cached langchain_text_splitters-0.3.9-py3-none-any.whl (33 kB)
  Using cached langsmith-0.4.8-py3-none-any.whl (367 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
  Using cached aiohttp-3.12.14-cp311-cp311-win_amd64.whl (452 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl (7.5 kB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install langchain pdfminer.six



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Extract and Clean PDF

In [3]:
from pdfminer.high_level import extract_text
import re

pdf_path = "HSC26-Bangla1st-Paper.pdf"
raw = extract_text(pdf_path)
def clean(text):
    t = re.sub(r'[a-zA-Z0-9]', '', text)
    t = re.sub(r'[^\u0980-\u09FF\s।,?!]', '', t)
    return re.sub(r'\s+', ' ', t).strip()
cleaned = clean(raw)
print(cleaned[:500])


িাাং া ১ম পত্র আল ািয রিষয় অপরিরিতা অনলাইন ব্যাচ সম্পর্কি ত যেককাকনা জিজ্ঞাাসা , র্িখনফল র্নম্নর্ব্ি ব্যজক্তি হঠাৎ র্ব্িিালী হক ওঠাি ফকল সমাকি পর্িচ সংকট সম্পককি ধািণা লাভ কিকব্। তৎকালীন সমািসভযতা ও মানব্তাি অব্মাননা সম্পককি িানকত পািকব্। তৎকালীন সমাকিি পণপ্রথাি কু প্রভাব্ সম্পককি িানকত পািকব্। তৎকাকল সমাকি ভদ্রকলাককি স্বভাব্বব্র্িষ্ট্য সম্পককি জ্ঞাানলাভ কিকব্। নািী যকামল ঠিক, র্কন্তু দুব্িল ন কলযাণীি িীব্নচর্িত দ্বািা প্রর্তজিত এই সতয অনুধাব্ন কিকত পািকব্। মানুষ আিা র্নক যব্ঁকচ থাকক অনুপকমি দৃষ


### Chunk For Q and A Pairs


In [4]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter

# (Optional) Process additional text with RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=200,
    separators=["।"]
)
chunks = splitter.split_text(cleaned)  # Assuming 'cleaned' is your input text

# Load the curated Q&A pairs from the JSON file
with open("bangla_english_qa_dataset.json", "r", encoding="utf-8") as f:
    qa_data_json = json.load(f)

# Initialize qa_data with the 68 curated pairs (34 Bangla + 34 English)
qa_data = qa_data_json["bangla_qa_pairs"] + qa_data_json["english_qa_pairs"]

# (Optional) Append chunk-based Q&A pairs if additional text needs to be covered
for c in chunks:
    qa_data.append({
        "context": c,
        "question": "এই অংশে কি লেখা হয়েছে?",
        "answer": c[:200]
    })

# Print dataset summary
print(f"Total Q&A pairs: {len(qa_data)}")
for i, qa in enumerate(qa_data[:5]):  # Show first 5 pairs for inspection
    print(f"Pair {i+1}:")
    print(f"Context: {qa['context']}")
    print(f"Question: {qa['question']}")
    print(f"Answer: {qa['answer']}\n")

Total Q&A pairs: 206
Pair 1:
Context: আমার বয়স সাতার মাত্র। এই জীবনটা না দদীঘিযি হাসাবে ব্যে, না গুনি হাসাবে। তবু ইহার একটু বিশেষ মূল্য আছে। ইহা যেই ফুলের মতা যাহার বুক্কি উপরি ভ্রমর আর্স া ব্র্স ারিল, এবং যেই পদক্ষেপি ইতিহাস তাহার জীবনের মাঝখানে ফুলের মতা গুটি ধরিয়া উঠি াছে।
Question: অনুপম তার জীবনের মূল্য সম্পর্কে কী বলেছেন?
Answer: অনুপম বলেছেন যে তার জীবন দীর্ঘ বা গুণে হাসাবে নয়, তবে এর একটু বিশেষ মূল্য আছে, যেমন একটি ফুলের মতো যার উপর ভ্রমর এসেছে এবং যার জীবনের মাঝখানে ইতিহাস ফুলের মতো গুটি ধরেছে।

Pair 2:
Context: অনুপমের বন্ধু হরিশ কানপুরি কাশি করে।
Question: অনুপমের বন্ধু হরিশ কী পেশায় নিয়োজিত ছিলেন?
Answer: অনুপমের বন্ধু হরিশ কানপুরে কাশি করতেন।

Pair 3:
Context: অনুপমের মামা বলেছিলেন, 'এটা আপনাদের লজ্জনস, আপনাদের কাশেই থাক।' এই কথাটি কন্যার গহনা পরীক্ষা করার সময় বলা হয়েছিল।
Question: অনুপমের মামা কেন বলেছিলেন, 'এটা আপনাদের লজ্জনস, আপনাদের কাশেই থাক'?
Answer: অনুপমের মামা এ কথা বলেছিলেন কারণ তিনি কন্যার গহনা পরীক্ষা করে দেখেন যে, সেটি খাঁটি সোনার নয়, এবং এমন নিম্নমানের গহনা গ

### Prepare Dataset

In [5]:
from datasets import Dataset

# Convert to HuggingFace Dataset
dataset = Dataset.from_list(qa_data)

# Check sample
print(dataset[0])


{'context': 'আমার বয়স সাতার মাত্র। এই জীবনটা না দদীঘিযি হাসাবে ব্যে, না গুনি হাসাবে। তবু ইহার একটু বিশেষ মূল্য আছে। ইহা যেই ফুলের মতা যাহার বুক্কি উপরি ভ্রমর আর্স া ব্র্স ারিল, এবং যেই পদক্ষেপি ইতিহাস তাহার জীবনের মাঝখানে ফুলের মতা গুটি ধরিয়া উঠি াছে।', 'question': 'অনুপম তার জীবনের মূল্য সম্পর্কে কী বলেছেন?', 'answer': 'অনুপম বলেছেন যে তার জীবন দীর্ঘ বা গুণে হাসাবে নয়, তবে এর একটু বিশেষ মূল্য আছে, যেমন একটি ফুলের মতো যার উপর ভ্রমর এসেছে এবং যার জীবনের মাঝখানে ইতিহাস ফুলের মতো গুটি ধরেছে।'}


### Load Pretrained LLM and Tokenizer

In [6]:
!pip install hf_xet

  Using cached hf_xet-1.1.5-cp37-abi3-win_amd64.whl (2.7 MB)



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "google/flan-t5-base"  # multilingual-compatible
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


###  Prepare Tokenization Function

In [8]:
def tokenize(batch):
    prompt = f"প্রসঙ্গ: {batch['context']} প্রশ্ন: {batch['question']} উত্তর:"
    inputs = tokenizer(prompt, truncation=True, max_length=512, padding="max_length")
    labels = tokenizer(batch["answer"], truncation=True, max_length=128, padding="max_length")

    inputs["labels"] = labels["input_ids"]
    return inputs

tokenized_dataset = dataset.map(tokenize, batched=False)


Map:   0%|          | 0/206 [00:00<?, ? examples/s]

### PEFT LoRA Setup

In [9]:
!pip install peft==0.9.0 transformers==4.36.2 accelerate bitsandbytes datasets sentencepiece


  Using cached peft-0.9.0-py3-none-any.whl (190 kB)
  Using cached transformers-4.36.2-py3-none-any.whl (8.2 MB)
  Using cached sentencepiece-0.2.0-cp311-cp311-win_amd64.whl (991 kB)
  Using cached tokenizers-0.15.2-cp311-none-win_amd64.whl (2.2 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.2
    Uninstalling tokenizers-0.21.2:
      Successfully uninstalled tokenizers-0.21.2


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\rag_env\\Lib\\site-packages\\~okenizers\\tokenizers.pyd'
Check the permissions.


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
!pip install accelerate==0.30.1 --upgrade --force-reinstall


  Using cached accelerate-0.30.1-py3-none-any.whl (302 kB)
  Using cached numpy-2.3.2-cp311-cp311-win_amd64.whl (13.1 MB)
  Using cached packaging-25.0-py3-none-any.whl (66 kB)
  Using cached psutil-7.0.0-cp37-abi3-win_amd64.whl (244 kB)
  Using cached PyYAML-6.0.2-cp311-cp311-win_amd64.whl (161 kB)
  Using cached torch-2.7.1-cp311-cp311-win_amd64.whl (216.1 MB)
  Using cached huggingface_hub-0.33.5-py3-none-any.whl (515 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl (308 kB)
  Using cached filelock-3.18.0-py3-none-any.whl (16 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl (43 kB)
  Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
  Using cached networkx-3.5-py3-none-any.whl (2.0 MB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Using cached fsspec-2025.7.0-py3-none-any.whl (199 kB)
  Using cached requests-2.32.4-py3-none-any.whl (64 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
  U

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\rag_env\\Lib\\site-packages\\~afetensors\\_safetensors_rust.pyd'
Check the permissions.


[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from peft import LoraConfig, get_peft_model
from transformers import BitsAndBytesConfig, AutoModelForSeq2SeqLM

# Use quantization
bnb_config = BitsAndBytesConfig(load_in_8bit=True)

# Load quantized base model
base_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=bnb_config)

# LoRA config
lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_2_SEQ_LM"
)

# Wrap with PEFT
model = get_peft_model(base_model, lora_config)

model.print_trainable_parameters()


The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers and GPU quantization are unavailable.


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3561


### Trainer Setup

In [12]:

#!pip install transformers==4.36.2 peft==0.9.0 accelerate bitsandbytes --upgrade


In [13]:
#!pip install transformers==4.36.2 peft==0.9.0 accelerate==0.25.0 bitsandbytes datasets sentencepiece --upgrade


In [14]:
from transformers import Seq2SeqTrainer, TrainingArguments, DataCollatorForSeq2Seq, GenerationConfig

# ✅ Subclass TrainingArguments to inject generation_config and avoid AttributeError
class MyTrainingArguments(TrainingArguments):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.generation_config = GenerationConfig()  # Dummy config to prevent error

# ✅ Setup training arguments using the custom class
training_args = MyTrainingArguments(
    output_dir="./finetuned_bangla_rag_model",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    learning_rate=3e-4,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    report_to="none"
)

# ✅ Setup data collator for padding sequences during training
collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# ✅ Create the Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,  # still works for now; will be deprecated in v5.0
    data_collator=collator,
)

# ✅ (Optional) You can still explicitly set the model's generation config if needed
trainer.model.generation_config = model.generation_config


C:\Users\User\AppData\Local\Temp\ipykernel_9112\2171403751.py:26: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


### Train and Save Model

In [15]:
import accelerate
import transformers

print("Accelerate version:", accelerate.__version__)
print("Transformers version:", transformers.__version__)


Accelerate version: 1.9.0
Transformers version: 4.53.3


In [16]:
import sys
'accelerate' in sys.modules


True

In [ ]:
trainer.train()
trainer.save_model()


c:\Users\User\rag_env\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss


### Test Inference From Fine Tuned Model

In [ ]:
from transformers import pipeline

pipe = pipeline("text2text-generation", model="./finetuned_bangla_rag_model", tokenizer=tokenizer)

# Test
query = "প্রসঙ্গ: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে? প্রশ্ন: অনুপমের ভাষায় সুপুরুষ কাকে বলা হয়েছে? উত্তর:"
print(pipe(query, max_new_tokens=100)[0]['generated_text'])
